# 1. Import needed libraries

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
#---------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
#---------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#---------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [ ]:
img_rows, img_cols = 128, 128
input_shape = (img_rows, img_cols, 3)

#n_classes = df['category'].nunique()
n_classes = 4
print('Total number of unique categories:', n_classes)

from os import listdir, makedirs
from os.path import isfile, join, basename, splitext, isfile, exists

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

import tensorflow as tf
import keras.backend as K

import keras
from keras.models import Sequential, Model
from keras.layers import Dropout, Dense, Flatten, BatchNormalization
from keras.layers import DepthwiseConv2D, SeparableConvolution2D, Convolution2D, Conv2D,GRU, LSTM, AlphaDropout, Embedding, ZeroPadding2D,AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dropout
from keras.layers import Concatenate, Average, Maximum, Bidirectional, TimeDistributed
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
#from keras.engine.input_layer import Input
from keras.models import load_model
#from keras.initializers import LecunNormal

import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option('precision', 30)
np.set_printoptions(precision = 30)


#tf.set_random_seed(1090)

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as img

import cv2
import itertools
import pathlib
import warnings
from PIL import Image
from random import randint
warnings.filterwarnings('ignore')

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef as MCC
from sklearn.metrics import balanced_accuracy_score as BAS
from sklearn.metrics import classification_report, confusion_matrix


from tensorflow import keras
from keras import layers
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory
##from keras.utils.vis_utils import plot_model
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from tensorflow.keras.layers import SeparableConv2D, BatchNormalization, GlobalAveragePooling2D

from distutils.dir_util import copy_tree, remove_tree

import os
#print(os.listdir("../input/alzheimer-mri-dataset/Dataset"))
import tensorflow as tf
from keras.datasets import mnist
import cv2
import os
import pathlib
from keras.layers import Conv2D, Conv2DTranspose,Concatenate, Dropout, Dense, Reshape, LayerNormalization, LeakyReLU
from keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, recall_score, precision_score
print("TensorFlow Version:", tf.__version__)

### Uncertainty Quantification of DRIFA-Net

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import KLDivergence

# Set a random seed for reproducibility
np.random.seed(42)

def create_ensemble(num_models, input_shape=(128, 128, 3)):
    ensemble_models = []
    
    for _ in range(num_models):
        model = DRIFA_Net(input_shape)  # Assuming ResNet18 is defined elsewhere
        ensemble_models.append(model)
    
    return ensemble_models

# Function to perform Monte Carlo Dropout inference

# Example usage
input_shape = (128, 128, 3)
num_models = 5
dropout_rate = 0.25

ensemble_models = create_ensemble(num_models, input_shape)

# Train each model in the ensemble
for i, model in enumerate(ensemble_models):
    print("Training Model", i)
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.compile(optimizer='adam', loss=[adaptive_knowledge_distillation_loss,
                                              adaptive_knowledge_distillation_loss,
                                             adaptive_knowledge_distillation_loss,
                                             adaptive_knowledge_distillation_loss], 
                      metrics=['accuracy', 'accuracy', 'accuracy', 'accuracy'])

    
    # Define checkpoint callback for each model
    checkpoint = ModelCheckpoint(f"best_student_models1_covid_brain_{i}.keras", monitor='val_loss', 
                                 verbose=1, save_best_only=True, mode='min')
    
    model.fit(x = [images_train_brain_mri, images_train_brain_ct, images_train_covid_cxr,  images_train_covid_ct], 
    y=([soft_prob_brain_mri, soft_prob_brain_ct, soft_prob_covid_cxr, soft_prob_covid_ct]),
    epochs=200,
    #validation_data=([X_val, X_val_c], [y_val, y_val_c]), 
    callbacks = [checkpoint],#batch_size=16,
    #validation_split = 0.2
    validation_data = ([X_val_brain_mri, X_val_brain_ct, X_val_covid_cxr, X_val_covid_ct],
                      [y_val_brain_mri, y_val_brain_ct, y_val_covid_cxr, y_val_covid_ct]), verbose=0)

In [ ]:
for i, model in enumerate(ensemble_models):
    model.evaluate([X_test_brain_mri, X_test_brain_ct,X_test_covid_cxr, X_test_covid_ct], 
[y_test_brain_mri,y_test_brain_ct, y_test_covid_cxr, y_test_covid_ct])

In [ ]:
'''def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=3):
    predictions = np.zeros((num_samples,) + model.predict([x1, x2, x3, x4],
                                                          verbose = 0).shape)
    print(len(predictions))
    
    for i in range(num_samples):
        print(i)
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose = 0)

    return predictions
'''

def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=30):
    predictions_shape = model.predict([x1, x2, x3, x4], verbose=0)[0].shape
    predictions = np.zeros((num_samples,) + predictions_shape)
    
    for i in range(num_samples):
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose=0)[0]

    return predictions
# Perform Monte Carlo Dropout inference for each model in the ensemble
ensemble_predictions = []
for model in ensemble_models:
    predictions = monte_carlo_dropout_inference(model, X_test_brain_mri, 
                                                        X_test_brain_ct,X_test_covid_cxr, X_test_covid_ct)
    ensemble_predictions.append(predictions)

#print('ensemble_predictions:', ensemble_predictions.shape)
# Take the mean or other aggregation method across the ensemble predictions



In [ ]:
final_prediction_model1 = np.mean(ensemble_predictions[0], axis=0)

# Use the final_prediction for further analysis or decision making
print("final_prediction_model1:", final_prediction_model1.shape)

In [ ]:
'''def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=3):
    predictions = np.zeros((num_samples,) + model.predict([x1, x2, x3, x4],
                                                          verbose = 0).shape)
    print(len(predictions))
    
    for i in range(num_samples):
        print(i)
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose = 0)

    return predictions
'''

def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=30):
    predictions_shape = model.predict([x1, x2, x3, x4], verbose=0)[1].shape
    predictions = np.zeros((num_samples,) + predictions_shape)
    
    for i in range(num_samples):
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose=0)[1]

    return predictions
# Perform Monte Carlo Dropout inference for each model in the ensemble
ensemble_predictions = []
for model in ensemble_models:
    predictions = monte_carlo_dropout_inference(model, X_test_brain_mri, 
                                                        X_test_brain_ct,X_test_covid_cxr, X_test_covid_ct)
    ensemble_predictions.append(predictions)

#print('ensemble_predictions:', ensemble_predictions.shape)
# Take the mean or other aggregation method across the ensemble predictions


In [ ]:
final_prediction_model2 = np.mean(ensemble_predictions[0], axis=0)

# Use the final_prediction for further analysis or decision making
print("final_prediction_model2:", final_prediction_model2.shape)

In [ ]:
'''def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=3):
    predictions = np.zeros((num_samples,) + model.predict([x1, x2, x3, x4],
                                                          verbose = 0).shape)
    print(len(predictions))
    
    for i in range(num_samples):
        print(i)
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose = 0)

    return predictions
'''

def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=30):
    predictions_shape = model.predict([x1, x2, x3, x4], verbose=0)[2].shape
    predictions = np.zeros((num_samples,) + predictions_shape)
    
    for i in range(num_samples):
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose=0)[2]

    return predictions
# Perform Monte Carlo Dropout inference for each model in the ensemble
ensemble_predictions = []
for model in ensemble_models:
    predictions = monte_carlo_dropout_inference(model, X_test_brain_mri, 
                                                        X_test_brain_ct,X_test_covid_cxr, X_test_covid_ct)
    ensemble_predictions.append(predictions)

#print('ensemble_predictions:', ensemble_predictions.shape)
# Take the mean or other aggregation method across the ensemble predictions


In [ ]:
final_prediction_model3 = np.mean(ensemble_predictions[0], axis=0)

# Use the final_prediction for further analysis or decision making
print("final_prediction_model3:", final_prediction_model3.shape)

In [ ]:
'''def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=3):
    predictions = np.zeros((num_samples,) + model.predict([x1, x2, x3, x4],
                                                          verbose = 0).shape)
    print(len(predictions))
    
    for i in range(num_samples):
        print(i)
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose = 0)

    return predictions
'''

def monte_carlo_dropout_inference(model, x1, x2, x3, x4, num_samples=30):
    predictions_shape = model.predict([x1, x2, x3, x4], verbose=0)[3].shape
    predictions = np.zeros((num_samples,) + predictions_shape)
    
    for i in range(num_samples):
        predictions[i, :] = model.predict([x1, x2, x3, x4], verbose=0)[3]

    return predictions
# Perform Monte Carlo Dropout inference for each model in the ensemble
ensemble_predictions = []
for model in ensemble_models:
    predictions = monte_carlo_dropout_inference(model, X_test_brain_mri, 
                                                        X_test_brain_ct,X_test_covid_cxr, X_test_covid_ct)
    ensemble_predictions.append(predictions)

#print('ensemble_predictions:', ensemble_predictions.shape)
# Take the mean or other aggregation method across the ensemble predictions


In [ ]:
final_prediction_model4 = np.mean(ensemble_predictions[0], axis=0)

# Use the final_prediction for further analysis or decision making
print("final_prediction_model4:", final_prediction_model4.shape)

In [ ]:
y_pred_binary_pgd_test1 = np.argmax(final_prediction_model1, axis=1)
y_pred_binary_pgd_test2 = np.argmax(final_prediction_model2, axis=1)
y_pred_binary_pgd_test3 = np.argmax(final_prediction_model3, axis=1)
y_pred_binary_pgd_test4 = np.argmax(final_prediction_model4, axis=1)

y_test_categorical1 = y_test_brain_mri
y_test_categorical2 = y_test_brain_ct
y_test_categorical4 = y_test_covid_ct
y_test_categorical3 = y_test_covid_cxr

y_test_categorical1 = np.argmax(y_test_categorical1, axis=1)
y_test_categorical2 = np.argmax(y_test_categorical2, axis=1)
y_test_categorical3 = np.argmax(y_test_categorical3, axis=1)
y_test_categorical4 = np.argmax(y_test_categorical4, axis=1)

## Task 1:
print('Task 1:')
print('Brain Tumours classification in MRI images:')
accuracy = accuracy_score(y_pred_binary_pgd_test1, y_test_categorical1) * 100
precision = precision_score(y_pred_binary_pgd_test1, y_test_categorical1, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test1, y_test_categorical1, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test1, y_test_categorical1, average='macro') * 100
#auc = roc_auc_score(y_pred, y_train_categorical, multi_class='ovr') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)

## Task 2:
print('Task 2:')
print('Brain Stroke classification in CT scan images:')
accuracy = accuracy_score(y_pred_binary_pgd_test2, y_test_categorical2) * 100
precision = precision_score(y_pred_binary_pgd_test2, y_test_categorical2, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test2, y_test_categorical2, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test2, y_test_categorical2, average='macro') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)

## Task 3:
print('Task 3:')
print('COVID19 classification in CXR images:')
accuracy = accuracy_score(y_pred_binary_pgd_test3, y_test_categorical3) * 100
precision = precision_score(y_pred_binary_pgd_test3, y_test_categorical3, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test3, y_test_categorical3, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test3, y_test_categorical3, average='macro') * 100
#auc = roc_auc_score(y_pred, y_train_categorical, multi_class='ovr') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)

## Task 4:
print('Task 4:')
print('COVID19 classification in CT scan images:')
accuracy = accuracy_score(y_pred_binary_pgd_test4, y_test_categorical4) * 100
precision = precision_score(y_pred_binary_pgd_test4, y_test_categorical4, average='macro') * 100
recall = recall_score(y_pred_binary_pgd_test4, y_test_categorical4, average='macro') * 100
f1 = f1_score(y_pred_binary_pgd_test4, y_test_categorical4, average='macro') * 100
print('accuracy:', accuracy)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)